In [52]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import pinecone
# from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

Extract data from the PDF

In [18]:
#Extract data from the PDF

def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls = PyPDFLoader)
    documents = loader.load()
    return documents

In [19]:
extracted_data = load_pdf("E:\projects\Medical-Chatbot-Project-\data")

Create text chunk

In [20]:
#Create text chunk
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [21]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


text_chunks

download embedding model

In [22]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [23]:
embeddings = download_hugging_face_embeddings()

In [24]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False)

In [25]:
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))


Length 384


In [26]:
def create_db(text_chunks, embeddings):
    DB_FAISS_PATH = 'vectorstore/db_faiss'
    
    db = FAISS.from_documents(text_chunks, embeddings)
    db.save_local(DB_FAISS_PATH)

In [27]:
create_db(text_chunks, embeddings)

In [58]:
# Load the FAISS database with the embeddings
db = FAISS.load_local("vectorstore/db_faiss", embeddings=embeddings)

# Encode your query text
query_text = "What are the symptoms of pneumonia?"

docs=db.similarity_search(query_text, k=3)
print(docs)

[Document(page_content='In the short term, there is the danger of pneumonia', metadata={'source': 'E:\\projects\\Medical-Chatbot-Project-\\data\\Medical_book.pdf', 'page': 567}), Document(page_content='• painful, bluish or purplish fingers or toes\n• puffy or swollen eyelids, face, feet, or lower legs\n• changes in the color of the face• skin rash, itching , or hives\n• yellow eyes or skin\n• severe or continuing headache\n• sore throat and fever, with or without chills• breathing problems or wheezing\n• tightness in the chest\n• dizziness\n• unusual tiredness or weakness• weight gain\nIn addition, patients taking anticoagulant drugs', metadata={'source': 'E:\\projects\\Medical-Chatbot-Project-\\data\\Medical_book.pdf', 'page': 265}), Document(page_content='Symptoms\nThe symptoms of the above disorders include:\n• Systemic lupus erythematosus. Symptoms include\nfever , chills, fatigue , weight loss, skin rashes (particu-', metadata={'source': 'E:\\projects\\Medical-Chatbot-Project-\\da

In [48]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [49]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}

In [54]:
llm=CTransformers(model=r"E:\projects\Medical-Chatbot-Project-\model\llama-2-7b-chat.ggmlv3.q2_K.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [55]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=db.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [57]:
while True:
    # user_input=input(f"Input Prompt:")
    user_input = f"Acne"
    result=qa({"query": user_input})
    print("Response : ", result["result"])

Response :  Acne is a skin condition that occurs when pores or oil glands in the face become inflamed, leading to blemishes and breakouts on the skin.
Response :  For moderate to severe acne, topical treatments such as benzoyl peroxide or tretinoin may be used. Isotretinoin (Accutane) is reserved for very severe cases of acne that do not respond to other treatments.
Response :  Use of benzoyl peroxide or tretinoin lotions, soaps, gels, and creams to clear up mild to moderately severe acne.
Response :  Use the following pieces of information to answer the user's question. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Acne is a skin disorder that affects the face and can be caused by a variety of factors, including hormonal imbalances and stress.
It can be treated with topical creams or gels containing benzoyl peroxide or tretinoin, as well as oral antibiotics in severe cases.
Isotretinoin (Accutane) is a powerful oral medication that is rese

KeyboardInterrupt: 